In [1]:
import pandas as pd
import numpy as np
import pickle
import sys
sys.path.append("../")
import rsc.Resources_New as rsc
from selenium import webdriver
from time import sleep
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from fuzzywuzzy import process, fuzz
import warnings
warnings.filterwarnings('ignore')

In [2]:
#Here i the csv that ill be using.
df_movies = pd.read_csv("../Data/imdb_top_1000.csv")

#I import the list that i scrapped with seleneum thats saved in a pickle so i dont lose the info when the kernel shuts down.
with open("../Data/libros_scrap.pkl", "rb") as alias:
    libro2 =pickle.load(alias)
with open("../Data/libros_quotes.pkl","rb") as alias:
    libros_quotes = pickle.load(alias)

In [3]:
#Seleneum configurations.

opciones= Options()
opciones.add_experimental_option('excludeSwitches', ['enable-automation'])#para ocultarme como robot
opciones.add_experimental_option('useAutomationExtension', False)
opciones.add_argument('--start-maximized') #empezar maximizado
opciones.add_argument('user.data-dir=selenium') #guarda las cookies
opciones.add_argument('--incognito')#incognito window


In [ ]:
#This loop runs my seleneum with its function.
libro = []
resultado = []
for i in df_movies["Series_Title"]:
    rsc.busqueda(i,resultado,2)
    print(resultado)
    libro.append(resultado)
    resultado = []

In [4]:
#Here i run all of my cleaning functions.
scrap_limpio = rsc.listas_libros(libros_quotes)
lista_votos_limpio = [rsc.limpieza_votos(voto) for voto in scrap_limpio[3]]
nota_promedio_limpio = [rsc.promedio(rati) for rati in scrap_limpio[2]]
Autor_limpio = [rsc.Autor(auto) for auto in scrap_limpio[0]]
fuzzy_titulo = rsc.semejanza(scrap_limpio[4],list(df_movies["Series_Title"]))
df_movies["Gross"] = df_movies["Gross"].str.replace(',','')

In [6]:
#I create columns in my movie DataFrame so i can then match movies with books
df_movies["Existe_libro"] = fuzzy_titulo
df_movies["Autor_libro"] = Autor_limpio
df_movies["Nota_promedio_libro"] = nota_promedio_limpio
df_movies["Votos_libro"] = lista_votos_limpio
df_movies["Nota_promedio_libro"] = df_movies["Nota_promedio_libro"].apply(lambda x: x*2)
df_movies["image"] = scrap_limpio[1]
df_movies["Quote"] = scrap_limpio[5]

In [10]:
df_movies

,Poster_Link,Series_Title,Released_Year,Certificate,Runtime,Genre,IMDB_Rating,Overview,Meta_score,Director,...,Star3,Star4,No_of_Votes,Gross,Existe_libro,Autor_libro,Nota_promedio_libro,Votos_libro,image,Quote
0,https://m.media-amazon.com/images/M/MV5BMDFkYT...,The Shawshank Redemption,1994,A,142 min,Drama,9.3,Two imprisoned men bond over a number of years...,80.0,Frank Darabont,...,Bob Gunton,William Sadler,2343110,28341469,True,Stephen King,8.94,2505.0,https://images-na.ssl-images-amazon.com/images...,No Quote
1,https://m.media-amazon.com/images/M/MV5BM2MyNj...,The Godfather,1972,A,175 min,"Crime, Drama",9.2,An organized crime dynasty's aging patriarch t...,100.0,Francis Ford Coppola,...,James Caan,Diane Keaton,1620367,134966411,True,Mario Puzo,8.76,392244.0,https://images-na.ssl-images-amazon.com/images...,“I'll make him an offer he can't refuse.”\n― M...
2,https://m.media-amazon.com/images/M/MV5BMTMxNT...,The Dark Knight,2008,UA,152 min,"Action, Crime, Drama",9.0,When the menace known as the Joker wreaks havo...,84.0,Christopher Nolan,...,Aaron Eckhart,Michael Caine,2303232,534858444,True,Frank Miller,8.52,180225.0,https://images-na.ssl-images-amazon.com/images...,“This should be agony. I should be a mass of a...
3,https://m.media-amazon.com/images/M/MV5BMWMwMG...,The Godfather: Part II,1974,A,202 min,"Crime, Drama",9.0,The early life and career of Vito Corleone in ...,90.0,Francis Ford Coppola,...,Robert Duvall,Diane Keaton,1129952,57300000,True,Elizabeth Tubbs,10.00,1.0,https://images-na.ssl-images-amazon.com/images...,No Quote
4,https://m.media-amazon.com/images/M/MV5BMWU4N2...,12 Angry Men,1957,U,96 min,"Crime, Drama",9.0,A jury holdout attempts to prevent a miscarria...,96.0,Sidney Lumet,...,Martin Balsam,John Fiedler,689845,4360000,True,Reginald Rose,8.14,29492.0,https://images-na.ssl-images-amazon.com/images...,“It takes a great deal of courage to stand alo...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,https://m.media-amazon.com/images/M/MV5BNGEwMT...,Breakfast at Tiffany's,1961,A,115 min,"Comedy, Drama, Romance",7.6,A young New York socialite becomes interested ...,76.0,Blake Edwards,...,Patricia Neal,Buddy Ebsen,166544,NaN,True,Truman Capote,7.72,246947.0,https://images-na.ssl-images-amazon.com/images...,"“Never love a wild thing, Mr. Bell,' Holly adv..."
996,https://m.media-amazon.com/images/M/MV5BODk3Yj...,Giant,1956,G,201 min,"Drama, Western",7.6,Sprawling epic covering the life of a Texas ca...,84.0,George Stevens,...,James Dean,Carroll Baker,34075,NaN,False,Roald Dahl,8.04,445692.0,https://images-na.ssl-images-amazon.com/images...,“I'd rather be fried alive and eaten by Mexica...
997,https://m.media-amazon.com/images/M/MV5BM2U3Yz...,From Here to Eternity,1953,Passed,118 min,"Drama, Romance, War",7.6,"In Hawaii in 1941, a private is cruelly punish...",85.0,Fred Zinnemann,...,Deborah Kerr,Donna Reed,43374,30500000,True,Caitlin Doughty,8.58,34111.0,https://images-na.ssl-images-amazon.com/images...,"“Insist on going to the cremation, insist on g..."
998,https://m.media-amazon.com/images/M/MV5BZTBmMj...,Lifeboat,1944,NaN,97 min,"Drama, War",7.6,Several survivors of a torpedoed merchant ship...,78.0,Alfred Hitchcock,...,Walter Slezak,William Bendix,26471,NaN,False,Mitch Albom,8.24,55251.0,https://images-na.ssl-images-amazon.com/images...,"“When someone passes, Benjamin, people always ..."


In [11]:
#I create a new DataFrame with only the rows that match that there is an existing book.
df_movies_limpio = df_movies[df_movies["Existe_libro"] == True]

#Here i copy all of the columns that i will use.
df_libros = df_movies_limpio[["Autor_libro","Nota_promedio_libro","Votos_libro"]].copy()

#And here i eliminate all of the columns that i won't be using anymore.

df_movies_limpio.drop("Existe_libro",inplace=True,axis=1),

df_movies_limpio["Quote"] = df_movies_limpio["Quote"].apply(lambda x: x.replace("\n", " "))

(None,)

In [34]:
df_movies_limpio["Autor_libro"].fillna("No info", inplace=True)
df_movies_limpio["Nota_promedio_libro"].fillna(0, inplace=True)
df_movies_limpio["Votos_libro"].fillna(0, inplace=True)

In [33]:
df_movies_limpio.count()

Poster_Link            659
Series_Title           659
Released_Year          659
Certificate            590
Runtime                659
Genre                  659
IMDB_Rating            659
Overview               659
Meta_score             572
Director               659
Star1                  659
Star2                  659
Star3                  659
Star4                  659
No_of_Votes            659
Gross                  554
Autor_libro            658
Nota_promedio_libro    658
Votos_libro            658
image                  659
Quote                  659
dtype: int64

In [31]:
df_movies_limpio["Nota_promedio_libro"].count()

658

In [35]:
#And finally i create the new CSV with all of the data that ill need for this project.
df_movies_limpio.to_csv(f"../Data/df_movies_limpio.csv")